# Using Simplicit's Easy API To Simulate Example Mesh
[Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/) is a mesh-free, representation-agnostic way to simulation elastic deformations. 

Here's a simple way to use the simplicit's code base. We can create a simple object, train it, simulate it and visualize all in a very few lines of code via our `easy_api`.

In [1]:
!pip install -q k3d
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import copy, math, os, sys, logging, threading
from typing import List, Tuple
from pathlib import Path

import numpy as np
import torch
import kaolin as kal

from IPython.display import display
from ipywidgets import Button, HBox, VBox

#local logger, prints at info or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

#logger used in the api code
logging.getLogger('kaolin.physics').setLevel(logging.INFO) # Prints everything at debug level or above


def print_tensor(t, name='', **kwargs):
    print(kal.utils.testing.tensor_info(t, name=name, **kwargs))

# Get the data 
Get data from [this onedrive link](https://nvidia-my.sharepoint.com/:f:/p/vismaym/EnH63QdIGP9LlPefq-2ydTMBr9felItBPvfI30WunpQlzw?e=f3qiQS)

In [ ]:
# set this to the path where you want to downloaded the mesh data
mesh_data_dir = "/home/vismaym/recode/TowakiSDFDataset/reconstructed_meshes/" 
meshes = {}
for mesh_file in os.listdir(mesh_data_dir):
    mesh_path = os.path.join(mesh_data_dir, mesh_file)
    meshes[mesh_file] = mesh_path

## Loading Geometry
Simplicits works with any geometry: meshes, pointclouds, SDFs, Gaussian splats, and more. For this example, we will use a mesh:

In [25]:
# Import and triangulate to enable rasterization; move to GPU
mesh = kal.io.obj.import_mesh(meshes["SDF_Towaki_Capsule.obj"], triangulate=True).cuda()
mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices.unsqueeze(0), normalize=True).squeeze(0) 
orig_vertices = mesh.vertices.clone()  # Also save original undeformed vertices
print(mesh)

SurfaceMesh object with batching strategy NONE
            vertices: [21812, 3] (torch.float32)[cuda:0]  
               faces: [43560, 3] (torch.int64)[cuda:0]  
       face_vertices: if possible, computed on access from: (faces, vertices)
        face_normals: if possible, computed on access from: (normals, face_normals_idx) or (vertex_normals, faces) or (vertices, faces)
            face_uvs: if possible, computed on access from: (uvs, face_uvs_idx)
      vertex_normals: if possible, computed on access from: (faces, face_normals)
     vertex_tangents: if possible, computed on access from: (faces, face_vertices, face_uvs, vertex_normals)
       vertex_colors: if possible, computed on access from: (faces, face_colors)
     vertex_features: if possible, computed on access from: (faces, face_features)
       face_tangents: if possible, computed on access from: (faces, vertex_tangents)
         face_colors: if possible, computed on access from: (faces, vertex_colors)
       face_features

## Sample Geometry
To enable simulation we need point samples within the object's volume, and physical material parameters per point. Lets set this up.

In [12]:
# Physics material parameters
soft_youngs_modulus = 1e5
poisson_ratio = 0.45
rho = 500  # kg/m^3
approx_volume = 0.5  # m^3

# Points sampled over the object's bounding box
num_samples = 100000
uniform_pts = torch.rand(num_samples, 3, device='cuda') * (orig_vertices.max(dim=0).values - orig_vertices.min(dim=0).values) + orig_vertices.min(dim=0).values
boolean_signs = kal.ops.mesh.check_sign(mesh.vertices.unsqueeze(0), mesh.faces, uniform_pts.unsqueeze(0), hash_resolution=512)

# use pts within the object
pts = uniform_pts[boolean_signs.squeeze()]
yms = torch.full((pts.shape[0],), soft_youngs_modulus, device="cuda")
prs = torch.full((pts.shape[0],), poisson_ratio, device="cuda")
rhos = torch.full((pts.shape[0],), rho, device="cuda")

import k3d
plot = k3d.plot()
plot += k3d.points(pts.cpu().detach().numpy(), point_size=0.01)
plot.display()

Output()

## Create and Train a SimplicitsObject
We encapsulate everything Simplicits method needs to know about the simulated object in a `SimplicitsObject` instance. Once the object is created, we need to run training to learn reduced degrees of freedom our simulator can use. 

**This will take a couple of minutes.** Please be patient.

In [13]:
# Initialize and train a Simpicits object to enable simulation
sim_obj = kal.physics.simplicits.SimplicitsObject(pts, yms, prs, rhos, torch.tensor([approx_volume], dtype=torch.float32, device="cuda"), num_handles=5)
print('Training simplicits object. This will take 2-3min. ')
sim_obj.train(num_steps=10000)  # TODO: with next patch add log_interval=1000
print('Object ready to simulate.')

# sim_obj.load_model('/tmp/test_easy_api.pt') # if you saved previously trained object, you can load it instead

Training simplicits object. This will take 2-3min. 
INFO:kaolin.physics.simplicits.easy_api:Training step: 0, le: 442.7199401855469, lo: 19170456.0
INFO:kaolin.physics.simplicits.easy_api:Training step: 1000, le: 603.0551147460938, lo: 41740.98046875
INFO:kaolin.physics.simplicits.easy_api:Training step: 2000, le: 662.5147705078125, lo: 43167.28515625
INFO:kaolin.physics.simplicits.easy_api:Training step: 3000, le: 577.317626953125, lo: 39980.375
INFO:kaolin.physics.simplicits.easy_api:Training step: 4000, le: 1514.6077880859375, lo: 1725.7955322265625
INFO:kaolin.physics.simplicits.easy_api:Training step: 5000, le: 893.8871459960938, lo: 2554.734130859375
INFO:kaolin.physics.simplicits.easy_api:Training step: 6000, le: 857.1710205078125, lo: 2087.34765625
INFO:kaolin.physics.simplicits.easy_api:Training step: 7000, le: 859.0032348632812, lo: 2339.7802734375
INFO:kaolin.physics.simplicits.easy_api:Training step: 8000, le: 1204.2548828125, lo: 5165.48779296875
INFO:kaolin.physics.simpli

In [ ]:
# Optionally, you can save/load this pre-trained object
# sim_obj.save_model('fox_mesh_model_10k_steps.pt')

## Create a Scene
Now we are ready to set up all the forces in the scene to simulated as well as simulation settings. For example, here we will add gravity and a floor plane.

In [14]:
scene = kal.physics.simplicits.SimplicitsScene() # default empty scene
#Convergence might not be guaranteed with few newton iterations, but runs very fast
scene.max_newton_steps = 3 

 The same `SimplicitsObject` can be added to multiple scene. Let's add it to our scene. Not we can reference it within the scene using `obj_idx`.

In [15]:
obj_idx = scene.add_object(sim_obj)

Lets set gravity and floor forces on the scene

In [16]:
scene.set_scene_gravity(acc_gravity=torch.tensor([0, 9.8, 0]))
scene.set_scene_floor(floor_height=-0.8, floor_axis=1, floor_penalty=1000)

We can play around with the material parameters of the object, indicated via `object_idx`

In [17]:
scene.set_object_materials(obj_idx, yms=torch.tensor(1e4, device='cuda', dtype=torch.float))

## Set Up Rendering
Let's set up rendering of our mesh so we can view it in a notebook.

In [26]:
def render(in_cam):
    active_pass = kal.render.easy_render.RenderPass.render
    render_res = kal.render.easy_render.render_mesh(in_cam, mesh)

    # use white background
    img = render_res[active_pass]
    background_mask = (
        render_res[kal.render.easy_render.RenderPass.face_idx] < 0).bool()
    img2 = torch.clamp(img, 0, 1)[0]
    img2[background_mask[0]] = 1
    final = (img2 * 255.).to(torch.uint8)
    return {"img": final, "face_idx": render_res[kal.render.easy_render.RenderPass.face_idx].squeeze(0).unsqueeze(-1)}

# faster low-res render during mouse motion
def fast_render(in_cam, factor=8):
    lowres_cam = copy.deepcopy(in_cam)
    lowres_cam.width = in_cam.width // factor
    lowres_cam.height = in_cam.height // factor
    return render(lowres_cam)


resolution = 512
camera = kal.render.easy_render.default_camera(resolution).cuda()
rest_state_viz = kal.visualize.IpyTurntableVisualizer(
    resolution, resolution, copy.deepcopy(camera), render, fast_render=fast_render,
    max_fps=24, world_up_axis=1)
rest_state_viz.show()

/home/vismaym/anaconda3/envs/sparsesimplicits/lib/python3.9/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Canvas(height=512, width=512)

Output()

## That's it! Let's Run and View or Physics Simulation
All we need to do now is run simulation and display the object using Kaolin's in-notebook visualizer.

In [27]:
# Reset mesh to its rest state
mesh.vertices = orig_vertices

fox_verts = []

global sim_thread_open, sim_thread
sim_thread_open = False
sim_thread = None

def run_sim():
    scene.reset()  # reset internal simultion state

    for s in range(50):
        with visualizer.out:
            scene.run_sim_step()
            print(".", end="")
        mesh.vertices = scene.get_object_deformed_pts(obj_idx, orig_vertices).squeeze()
        if s%10 == 0:
            fox_verts.append(mesh.vertices.clone().detach())
        visualizer.render_update()

def start_simulation(b):
    global sim_thread_open, sim_thread
    with visualizer.out:
        if(sim_thread_open):
            sim_thread.join()
            sim_thread_open = False
        sim_thread_open = True
        sim_thread = threading.Thread(target=run_sim, daemon=True)
        sim_thread.start()

scene.reset_object(obj_idx)
button = Button(description='Run Sim')
button.on_click(start_simulation)
visualizer = kal.visualize.IpyTurntableVisualizer(
    resolution, resolution, copy.deepcopy(camera), render, fast_render=fast_render,
    max_fps=24, world_up_axis=1)
visualizer.render_update()  # render first frame
display(HBox([visualizer.canvas, button]), visualizer.out)

Output()